# Differential and Functional Analysis of Metatranscriptome Data in R

## Before you begin:

Required Inputs:
    1. info.matrix
    2. design.matrix
    3. contrast.matrix
    4. all sample .cyto and .count files
    
There are many different species in the microbiome, many of which share the same genes, creating ambiguous read mapping. This ambiguity and the presence of potentially thousands of species necessitates requres different handling than single-organism RNA-seq data. The MUSCATO Pipeline includes a perl script that identifies the Last Common Ancestor (LCA) for each multi-mapping read, then generates an organism output (a .cyto file) and a gene output (a .counts file) that includes both the normalized (RPM) read counts and the longest read LCA per gene.

If you do not already have these, you can run the command: perl BOSS/BOSS.pl  and then when it asks if you want to get DEGs, answer yes. Then just run: qsub BOSS.pbs. It will only run the parts of the script necessary to get these input files.

## Inputting design, contrasts, and count data

### First thing to do is make sure that you are in the right directory and have all necessary R packages

In [10]:
setwd("R:\\RNASEQ\\TONSIL\\STATS")
library('data.table')

#input design matrix for limma
design.matrix = fread("design.matrix", header = TRUE, sep = "\t", stringsAsFactors = FALSE, na.strings="NA", data.table = FALSE)
row.names(design.matrix)=design.matrix[,1]
design.matrix[,1] = NULL

#input contrast matrix for limma
contrast.matrix = fread("contrast.matrix", header = TRUE, sep = "\t", stringsAsFactors = FALSE, na.strings="NA", data.table = FALSE)
row.names(contrast.matrix)=contrast.matrix[,1]
contrast.matrix[,1] = NULL

### Now input gene count files, containing LCAs and read counts

In [12]:
#get filenames
  files <- list.files("./", pattern="\\.counts", full.names=TRUE)
  fn = rownames(design.matrix)
  labs = fn
  labs = gsub("_.*", "", labs)
  filenames = character(length = length(fn))
  if(length(files)!=length(fn)){
        print("Some of the counts files are missing, type 'ls muscato/*counts' into the command line and make sure that they match what is in your design.matrix. Once that is done, type 'qsub muscato/checkjob.pbs'. Any other problems email tealfurn@umich.edu.")
        quit()}
  for( i in 1:length(fn) ){
    good = grep(fn[i], files, value = TRUE)
    filenames[i] <- good
  }
  rm(files)
  rm(good)
  rm(i)